<a href="https://colab.research.google.com/github/AyozeGS/IABD/blob/main/7RO/UT1/T2/7RO_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TAREA ALGORITMOS EVOLUTIVOS PROGRAMACIÓN EN IA
**EL PROBLEMA DEL VIAJANTE DE COMERCIO**

Crea un programa en Python que resuelva el problema para al menos 5 poblaciones (por ejemplo Telde, Gáldar, etc) usando el algoritmo evolutivo cuyas funciones de mutación y de recombinación vienen descritas en https://www.youtube.com/watch?v=3Kzj2FNaua8

**ENLACES DE INTERÉS:**
https://pythondiario.com/2018/05/el-problema-de-las-n-reinas-algoritmos.html
https://jarroba.com/algoritmos-geneticos-ejemplo
https://es.wikipedia.org/wiki/Problema_del_viajante

**RÚBRICA:**



*   (4 Puntos) El algoritmo muestra soluciones aceptables (para ello usa distancias reales en la matriz de distancias)
*   (4 Puntos) Se crean y comentan los métodos de mutación, fitness, y recombinanción.
*   (2 Puntos) Uso del vocabulario correcto (población, generación etc) así como explicación de los hiperparámetos (probabilidad de recombinación, de mutación, población inicial etc).

Se subirá un archivo PDF con el notebook de Jupiter incluyendo la ejecución y los markdown con las explicaciones.

In [ ]:
import pandas as pd
import numpy as np

#Load file and clean data
df = pd.read_csv("data_municipios.csv", index_col=0)
df[df.isna()] = 0
display(df)
distances = df.to_numpy(dtype=int)
municipalities = df.index.to_numpy()
indexes = np.array(range(len(municipalities)))

#params
n_population = 10
#vars
population = []
population_total_dist = []

,Las Palmas,Telde,Arucas,Galdar,Teror,Agaete,Mogan
Distancia(km),,,,,,,
Las Palmas,0.0,15.0,9.0,22.0,13.0,26.0,39.0
Telde,15.0,0.0,17.0,28.0,15.0,30.0,33.0
Arucas,9.0,17.0,0.0,13.0,7.0,18.0,33.0
Galdar,22.0,28.0,13.0,0.0,14.0,7.0,30.0
Teror,13.0,15.0,7.0,14.0,0.0,16.0,26.0
Agaete,26.0,30.0,18.0,7.0,16.0,0.0,24.0
Mogan,39.0,33.0,33.0,30.0,26.0,24.0,0.0


First Population

In [ ]:
population.clear()
population_total_dist.clear()

#Generate N random vectors from indexes. Ex: [4 0 5 2 6 1 3]
while len(population) < n_population:
  member = np.copy(indexes)
  np.random.shuffle(member)
  #Check if member already in population
  if not population or not (member == population).all(axis=1).any():
    population.append(member)

print(np.array(population))

#Calculate total distance for each member and save in an array
for member in population:
  sum = 0
  for i in range(len(member)-1):
    sum += df.iloc[member[i]][member[i+1]]
  population_total_dist.append(sum)


In [ ]:
print(population_total_dist)
min_index = population_total_dist.index(min(population_total_dist))
print("distancia minima:", population_total_dist[min_index], "km.")
for i in population[min_index]:
  print(df.index[i])

[145.0, 120.0, 133.0, 105.0, 97.0, 138.0, 138.0, 117.0, 135.0, 148.0]
distancia minima: 97.0 km.
Telde
Las Palmas
Galdar
Arucas
Teror
Agaete
Mogan


In [ ]:
a = np.array([[5, 6, 3, 2, 1, 4, 0],
 [6, 5, 3, 4, 1, 0, 2],
 [5, 4, 1, 0, 3, 6, 2],
 [0, 3, 1, 4, 2, 6, 5],
 [0, 1, 4, 3, 6, 5, 2],
 [3, 2, 5, 6, 1, 0, 4],
 [3, 2, 6, 0, 4, 1, 5],
 [2, 0, 1, 4, 5, 6, 3],
 [1, 4, 2, 3, 0, 6, 5],
 [2, 1, 3, 4, 6, 5, 0]])
b = np.array([3, 2, 5, 6, 0, 1, 4])

#print(np.any(member == np.all(population)))

(a == b).all(axis=1).any()

False